In [12]:
from math import ceil
import time
import re
import ast
import sys
import urllib
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.error import URLError
import requests
from itertools import cycle
import string
from header_list import user_agent_list
from proxies_list import proxies_list_
from FUNCTIONS import proxie_check, strip_count, gen_url_list
from SCRAPPER_FUNCTIONS import session_creator, each_page, links_for_props
from MAIN_FUNCTIONS import each_property
ua = user_agent_list
proxies = proxies_list_

In [13]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home: 


In [14]:
# customer_df = each_property(customer_url, header, proxy)

In [20]:
customer_df = pd.DataFrame(columns=['address', 'city', 'state', 'zip', 'price', 
                                    'size', 'num_beds', 'num_baths', 'lot_size',
                                    'yr_blt', 'HOA', 'HOA_fee'])

In [19]:
search_zip_code = customer_df['zip'].values
search_city = customer_df['city'].values
price = customer_df['price'].values
sqft = customer_df['sqft'].values
num_bds = customer_df['num_bds'].values
num_bths = customer_df['num_bths'].values
lot_sqft = customer_df['lot_sf'].values
yr_blt = customer_df['yr_blt'].values
hoa = customer_df['HOA'].values
hoa_fee = customer_df['HOA_fee'].values

if price<=50000:
    min_price = ''

# search_url = 'https://www.redfin.com/zipcode/'+customer_df['zip']+ \
# '/filter/sort=lo-days,property-type='+
# house+condo+townhouse
# ,min-price=
# 100k
# ,max-price=
# 1M
# ,min-beds=
# 3
# ,min-baths=
# 1
# ,min-sqft=
# 1.25k
# -sqft
# ,min-year-built=
# 1960
# ,min-lot-size=
# 6.5k-sqft
# ,hoa=
# 400


In [15]:
active_zip_url_list = gen_active_url_list(zip_)
df = pd.DataFrame(columns=['full_address', 'home_link'])
active_zip_url_list, df, proxies = links_for_active_props(proxies, active_zip_url_list, df, ua)
# for url in df['home_link']:
#     active_df = each_property(url, header, proxy)
# return active_df[0:2]